In [13]:
import pandas as pd
from pathlib import Path
import numpy as np

# ====== 1. Đường dẫn thư mục ======
data_dir = Path(r"C:\Downloads\Corn_data_merged_3_region")

p_condition = data_dir / "corn_condition_all_years_3region.csv"
p_progress  = data_dir / "corn_progress_all_years_3region.csv"
p_weather   = data_dir / "weather_weekly_all_years_3region.csv"
out_path    = data_dir / "corn_weekly_merged.csv"

# ====== 2. Đọc dữ liệu ======
condition = pd.read_csv(p_condition)
progress  = pd.read_csv(p_progress)
weather   = pd.read_csv(p_weather)

# ====== 3. Chuẩn hoá ======
def normalize(df):
    df = df.copy()
    if 'state_alpha' in df.columns:
        df['state_alpha'] = df['state_alpha'].astype(str).str.upper().str.strip()
    if 'region' in df.columns:
        df['region'] = df['region'].astype(str).str.strip()
    if 'week_ending' in df.columns:
        df['week_ending'] = pd.to_datetime(df['week_ending'], errors='coerce')
    return df

condition, progress, weather = map(normalize, [condition, progress, weather])

# ====== 4. Lọc WEATHER theo tuần của PROGRESS ======
keys = ['state_alpha', 'year', 'region', 'week_ending']
progress_weeks = progress[keys].drop_duplicates()
weather_f = weather.merge(progress_weeks, on=keys, how='inner')

# ====== 5. Gộp dữ liệu ======
cond_nonkeys = [c for c in condition.columns if c not in keys]
prog_nonkeys = [c for c in progress.columns  if c not in keys]
weath_nonkeys= [c for c in weather_f.columns if c not in keys]

df = (
    progress[keys + prog_nonkeys]
    .merge(condition[keys + cond_nonkeys], on=keys, how='left')
    .merge(weather_f[keys + weath_nonkeys], on=keys, how='left')
)

# ====== 6. Hợp nhất các cột *_x/_y nhưng KHÔNG đụng 'week_ending' ======
def coalesce_precise(df, base, candidates=None):
    """
    Gộp các biến thể của 'base' (vd: base_x, base_y) vào 1 cột 'base'.
    Nếu candidates=None, tự tìm base_x/base_y; KHÔNG động tới cột khác.
    """
    if candidates is None:
        candidates = [c for c in df.columns if c == base or c in (base + '_x', base + '_y')]
    else:
        candidates = [c for c in candidates if c in df.columns] + ([base] if base in df.columns else [])
    if not candidates:
        return df
    df[base] = df[candidates].bfill(axis=1).iloc[:, 0]
    for c in candidates:
        if c != base:
            df.drop(columns=c, inplace=True, errors='ignore')
    return df

# Gộp state_name (nếu có state_name_x/y)
df = coalesce_precise(df, 'state_name')

# TUYỆT ĐỐI không sờ vào 'week_ending' ngoài việc đảm bảo parse datetime
# (đã parse ở bước normalize). Nếu có 'week_ending_x/y' thì mới gộp:
cand_we = [c for c in ('week_ending_x','week_ending_y') if c in df.columns]
df = coalesce_precise(df, 'week_ending', candidates=cand_we)

# Gộp 'week' chỉ từ week_x/week_y (không match bằng startswith để tránh trùng với week_ending)
cand_w = [c for c in ('week_x','week_y') if c in df.columns]
if 'week' in df.columns or cand_w:
    df = coalesce_precise(df, 'week', candidates=cand_w)

# Ép kiểu cho chắc chắn
if 'week_ending' in df.columns:
    df['week_ending'] = pd.to_datetime(df['week_ending'], errors='coerce')
if 'week' in df.columns:
    df['week'] = pd.to_numeric(df['week'], errors='coerce').astype('Int64')

# ====== 7. Condition: fill 0 & flag chỉ dựa vào condition ======
cond_cols = [c for c in ['excellent','good','fair','poor','very poor'] if c in df.columns]
for c in cond_cols:
    df[c] = df[c].fillna(0)
df['flag_condition_progress'] = (df[cond_cols].fillna(0).sum(axis=1) == 0).astype(int)

# ====== 8. Thứ tự cột: week_ending trước week ======
base_cols = ['year', 'state_name', 'state_alpha', 'region']
if 'week_ending' in df.columns: base_cols.append('week_ending')
if 'week' in df.columns:        base_cols.append('week')
remaining = [c for c in df.columns if c not in base_cols]
df = df[base_cols + remaining]

# ====== 9. Kiểm tra & sắp xếp ======
assert 'week_ending' in df.columns, "week_ending bị thiếu!"
assert 'week' in df.columns, "week bị thiếu!"
sort_keys = ['state_alpha','year','region','week_ending']
df = df.sort_values([k for k in sort_keys if k in df.columns]).reset_index(drop=True)


df = df.sort_values(sort_keys).reset_index(drop=True)
df.to_csv(out_path, index=False)

print(f" Đã tạo file: {out_path}")
print(f"   Số dòng: {df.shape[0]} | Số cột: {df.shape[1]}")
print("   Flag = 1 (condition toàn 0):", int((df['flag_condition_progress']==1).sum()))


 Đã tạo file: C:\Downloads\Corn_data_merged_3_region\corn_weekly_merged.csv
   Số dòng: 11890 | Số cột: 25
   Flag = 1 (condition toàn 0): 3864
